Author: Rashad Haddad    
Description: In this notebook, using a reasonable set of assumptions, we seek a choice of parameters for our model so that it converges to the correct score over a period of time.  

In [2]:
from typing import Dict

import plotly.express as px

import pandas as pd
import numpy as np

import sys
if "../" not in sys.path:
    sys.path.append("../")
from lib.obligor import Obligor
from lib.credit_migration_schema import MigrationParams

In [3]:
N_borrows = 1000  # number of borrowers in our sim  
N_trials = 100  # number of eval points  
random_seed = 42  # ensure same results on repeated trials.   

r = 0.04  # 4% p.a.
vol = 0.8  # 80% p.a.
time_step = 1/365  # each time step is 1 day.

LTV_orig = 1 / 1.4  # origination ltv
LTV_liq = 1 / 1.2 # liquidation ltv

repay_prob = 0.85  # probability borrower repays debt in any period

In [4]:
# simulate borrows times
borrow_times = np.random.random(size=(N_borrows, N_trials+1))
borrow_times[:,0] = 1  # all start with a loan
borrow_times[:,1:] = np.where(borrow_times[:,1:] > 0.5, 1, 0)

# simulate repay times
repay_times = np.zeros(borrow_times.shape)
repay_times[:,1:] = np.where(np.random.random(size=repay_times[:,1:].shape)>(1-repay_prob), 1, 0)

# simulate repay amounts
repay_amts = np.where(repay_times > 0, np.ones(repay_times.shape), 0)

# simulate asset value following GBM
var = vol ** 2
T = N_trials * time_step
Z = np.random.normal(size=(1, N_trials+1))
Z[0,0] = 0  # must start at 0
drift = (T / N_trials) * np.arange(0, N_trials+1).reshape((1, N_trials+1))
asset_value = np.exp(r*drift + (r - 0.5*var)*drift + vol*(drift**0.5)*Z)

In [7]:
def process(X : tuple, borrow_times : np.ndarray, repay_times : np.ndarray, repay_amts : np.ndarray, asset_value : np.ndarray, start_alpha: int = 10, start_beta: int = 10) -> float:

    # upack the parameters
    c0, xi0 = X[0], X[1]    
    c1, xi1 = X[2], X[3]
    c2, xi2 = X[4], X[5]
    c3, xi3 = X[6], X[7]

    # set up migration params...
    migration_params = MigrationParams(c0=c0,xi0=xi0,c1=c1,xi1=xi1,c2=c2,xi2=xi2, c3=c3, xi3=xi3)

    # set up an array to track borrowers with the params
    borrower_collect: np.ndarray[Obligor] = np.ndarray(shape=(1, N_borrows), dtype=Obligor)
    for i in range(borrower_collect.shape[1]):
        borrower_collect[0,i] = Obligor(alpha=start_alpha, beta=start_beta,migration_params=migration_params)

    # set up array of alphas, betas to track results
    alphas = start_alpha * np.ones(borrow_times.shape)
    betas = start_beta * np.ones(repay_times.shape)
    
    # run the procedure...
    for j in range(alphas.shape[1]):
        
        for i in range(N_borrows):
            # apply origination where is borrow time
            borrower = borrower_collect[0,i]

            if borrow_times[i,j] > 0:
                borrower.add_borrow(amount=1, tenor=0, collateral_amt = 1 / (LTV_orig*asset_value[0,j]))

            perp_loan = borrower._fetch_loan()

            if not isinstance(perp_loan, type(None)):

                # apply repay where is repay time
                if repay_times[i,j] > 0:
                    
                    # set amount to repay
                    repay_amt = min(repay_amts[i,j], perp_loan.outstanding_amount)

                    # add repay for borrower
                    borrower.add_repay(amount=repay_amt, repayment_time = 0)

                    # we will assume the borrower get back
                    # or withdraws excess collateral
                    # compute this as the amount of collateral
                    # units that back 1 unit of the liability
                    excess_colat = (1 / (LTV_liq*asset_value[0,j]))
                    excess_colat = max(excess_colat, perp_loan.collateral_amt)

                    # withdraw excess collat
                    borrower.withdraw_collateral(withdraw_amt=excess_colat)

            perp_loan = borrower._fetch_loan()

            if not isinstance(perp_loan, type(None)):
                # apply liq where is liq time
                borrower_ltv = perp_loan.ltv(asset_value[0,j])

                if borrower_ltv > LTV_liq:

                    # compute untis collat to liq...
                    amt_to_liq = (LTV_liq*asset_value[0,j]*perp_loan.collateral_amt - perp_loan.outstanding_amount) / (LTV_liq*asset_value[0,j] - asset_value[0,j])
                    
                    # ensure the amount to liquidate is bounded
                    amt_to_liq = min(max(amt_to_liq,0), perp_loan.collateral_amt)

                    # add liquidation for borrower
                    borrower.add_liquidation(amt_to_liq=amt_to_liq, asset_price=asset_value[0,j], repayment_time = 0)

            # record current alpha and beta for further analysis
            alphas[i,j] = borrower._alpha
            betas[i,j] = borrower._beta

    # compute ending probs..
    ending_probs = np.divide(alphas[:,-1], alphas[:,-1] + betas[:,-1])

    # return the ending probability
    return ending_probs.mean()

# target function, minimize difference from target prob..
def target(X : tuple, target_prob : float = 0.85):

    # get the obs prob from + repay times
    obs_prob_plus = process(X, borrow_times = borrow_times, repay_times = repay_times, repay_amts = repay_amts, asset_value = asset_value, start_alpha = 10, start_beta = 10)

    # flip the repay times and do the same thing.  
    # if borrower is good 85% of the time, now they are 
    # good only 15% of the time...
    # do 1 - repay times to flip probability
    # recall repay amts is 1 when repay times is 1 else 0
    obs_prob_flip = process(X, borrow_times = borrow_times, repay_times = 1 - repay_times, repay_amts = 1 - repay_amts, asset_value = asset_value, start_alpha = 10, start_beta = 10)

    return (target_prob - obs_prob_plus)**2 + ((1-target_prob) - obs_prob_flip)**2

Test run

In [8]:
c0=1
xi0=171.8281828459045
c1=1
xi1=171.8281828459045
c2=1
xi2=171.8281828459045
c3=1
xi3=171.8281828459045


process((c0, xi0,c1, xi1,c2, xi2,c3, xi3,), borrow_times = borrow_times, repay_times = repay_times, repay_amts = repay_amts, asset_value = asset_value, start_alpha = 10, start_beta = 10)

0.33891484349966694